In [1]:
import utils
import feature_extractor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Constant Definition

In [8]:
NUM_CLASSES=23
BATCH_SIZE=64
EPOCHS=10
HIDDEN_SIZE=64
VECTOR_SIZE = 100  # Size of word vectors
WINDOW_SIZE = 5  # Context window size
THREADS = 4  # Number of threads to use for training
CUTOFF_FREQ = 1  # Minimum frequency for a word to be included in vocabulary
TRAINING_SIZE = 10000  
TEST_SIZE = 10

# reading dataset

In [3]:
complete_data=utils.read_data("../data/fixed_PIZZA_train.json")

In [9]:
data = complete_data[:TRAINING_SIZE]
corpus, top, decoupled = utils.get_train_dataset(data)

In [5]:
entites_output_as_number_labels,intents_output_as_number_labels, input_as_tokenized_string=utils.label_complete_input(corpus, decoupled, top)

# Embedding model

In [6]:
emb_model = feature_extractor.load_pretrained_model('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [17]:
# emb_model.most_similar('to')
emb_model['pizza']

array([ 0.22842  ,  0.045378 , -0.55545  , -0.0032376, -0.037485 ,
        0.46291  ,  0.99807  , -0.50831  , -0.15604  ,  0.28439  ,
       -0.27876  , -0.24536  ,  0.63619  ,  0.50568  ,  0.023984 ,
       -0.2106   ,  0.22656  , -0.23071  , -0.54049  ,  0.6044   ,
        0.33611  ,  0.89848  , -0.34049  , -0.80834  ,  0.048194 ,
        0.996    ,  0.074092 ,  0.10669  , -0.39791  ,  0.058263 ,
        0.54617  ,  0.64601  ,  0.76199  , -0.38709  ,  0.3407   ,
        1.0066   ,  0.055354 , -0.4413   ,  0.59654  , -1.5691   ,
        0.91143  , -0.24049  , -0.13928  , -0.79321  , -0.015157 ,
        0.14756  , -0.81427  , -0.64968  ,  0.43528  ,  0.44417  ,
       -0.36759  ,  0.13749  , -0.27889  ,  0.59873  , -1.002    ,
       -1.0241   , -0.14981  ,  0.94957  ,  0.55613  ,  0.54161  ,
       -0.42752  ,  0.17997  ,  0.3582   ,  0.15007  , -0.043267 ,
       -0.72917  , -0.19583  , -0.072447 ,  0.08813  , -0.3302   ,
       -0.076771 , -0.066435 ,  0.6275   ,  0.21399  ,  0.1754

# NER Model

In [13]:
class LargeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data 
        self.labels = labels 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    #I believe we can transform words into embeddings here
    embeddings=[]
    # print(sequences)
    for seq in sequences:
        x=[]
        for token in seq:
            x.append(emb_model[token])
        embeddings.append(x)
    sequences=embeddings
    labels = [torch.tensor(label, dtype=torch.long) for label in labels]
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, padded_labels, lengths

class LargeWordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LargeWordRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.fc(out)
        return out

# Checking if CUDA is working

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available()) 
print(torch.version.cuda)
print(f'Using device: {device}')

False
None
Using device: cpu


# Training Model

In [15]:


labels = entites_output_as_number_labels

dataset = LargeDataset(input_as_tokenized_string, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True, num_workers=0)

model = LargeWordRNN(input_size=VECTOR_SIZE, hidden_size=HIDDEN_SIZE, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(EPOCHS): 
    for padded_sequences, padded_labels, lengths in dataloader:
        padded_sequences=padded_sequences.to(device)
        padded_labels=padded_labels.to(device)
        lengths=lengths.to(device)
        optimizer.zero_grad()
        outputs = model(padded_sequences, lengths)
        loss = criterion(outputs.view(-1, NUM_CLASSES), padded_labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


KeyError: "Key 'peperoni' not present"

In [24]:
# test_data=complete_data[200000:200010]
# print(test_data)
# print("---------------------------------------------")
# test_corpus,_,_= utils.get_train_dataset(test_data)
# print(test_corpus)
# test_as_tokenized_string=feature_extractor.list_of_lists(test_corpus)

In [25]:
# print(test_as_tokenized_string)

In [26]:
# class TestLargeDataset(Dataset):
#     def __init__(self, data):
#         self.data = data 
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         return self.data[idx]

# def test_collate_fn(batch):
#     sequences = batch
#     #I believe we can transform words into embeddings here
#     embeddings=[]
#     for seq in sequences:
#         print(seq)
#         x=[]
#         for token in seq:
#             x.append(emb_model.wv[token])
#         embeddings.append(x)
#     sequences=embeddings
#     sequences = [torch.tensor(seq) for seq in sequences]
#     padded_sequences = pad_sequence(sequences, batch_first=True)
#     lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
#     return padded_sequences, lengths


# test_as_tokenized_string
# dataset = TestLargeDataset(test_as_tokenized_string)
# dataloader = DataLoader(dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)

# for padded_sequences, lengths in dataloader:
#     print("-------------------------------")
#     padded_sequences=padded_sequences.to(device)
#     lengths=lengths.to(device)
#     outputs = model(padded_sequences, lengths)
#     entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}
#     for i, out in enumerate(outputs[0]):
#         num = torch.argmax(out).int().item()
#         for key, value in entity_to_num.items():
#             if value == num:
#                 print(key)
#                 break


    
# #  entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}


In [27]:
    # entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}


In [ ]:
dev_data=utils.read_data("../data/fixed_PIZZA_dev.json")
dev_corpus, dev_top = utils.get_dev_dataset(dev_data)
gold_dev_labels,dev_as_tokenized_string=utils.label_complete_dev(dev_corpus, dev_top)

class TestLargeDataset(Dataset):
    def __init__(self, data):
        self.data = data 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def test_collate_fn(batch):
    sequences = batch
    #I believe we can transform words into embeddings here
    embeddings=[]
    for seq in sequences:
        print(seq)
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, lengths

dev_dataset = TestLargeDataset(dev_as_tokenized_string)
dataloader = DataLoader(dev_dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)
model_output=[]
for padded_sequences, lengths in dataloader:
    print("-------------------------------")
    labels = []
    padded_sequences=padded_sequences.to(device)
    lengths=lengths.to(device)
    outputs = model(padded_sequences, lengths)
    entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17,"I_NOT_STYLE": 18, "B_NOT_STYLE": 19, "B_NOT_QUANTITY": 20, "I_NOT_QUANTITY": 21, "NONE": 22}
    for i, out in enumerate(outputs[0]):
        num = torch.argmax(out).int().item()
        labels.append(num)
    model_output.append(labels)

confusion_matrix, accuracy=utils.calc_accuracy(model_output, gold_dev_labels)
print(confusion_matrix)
print(accuracy)

['i', 'want', 'to', 'order', 'two', 'medium', 'pizzas', 'with', 'sausage', 'and', 'black', 'olives', 'and', 'two', 'medium', 'pizzas', 'with', 'pepperoni', 'and', 'extra', 'cheese', 'and', 'three', 'large', 'pizzas', 'with', 'pepperoni', 'and', 'sausage']


KeyError: "Key 'to' not present"